# Prompts and instructions. Day 3.

## Agenda:
* Will learn how to create different prompts (e.g. instruction prompts) and how to enrioch our input text data with prompt instuctions.
* Will look how zero-shot, one-shot and few-shot prompts look like and how they can help LLM work better.
* Will explore the most impactful parameters of text generation and look up how generated text differs with various generation options.
* Llama 2 - a state of the art LLM will be used for our practice.






In [ ]:
!pip install -q accelerate==0.21.0 transformers==4.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.6 MB/s eta 0:00:00


In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    PretrainedModel,
    PreTrainedTokenizer,
    PreTrainedTokenizerFast,
)
import torch
import pandas as pd

## Load Llama2 chat model

We will use a chat version of Llama-2 model that was optimised for dialogue use cases.
Read more about the model https://huggingface.co/NousResearch/Llama-2-7b-hf


In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)

The model is very big and it may take about 5 minutes to download it. Make sure your GPU runtime is on, you will need GPU to use that model.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def generate_text(model: PretrainedModel, prompt: str) -> None:
    """
    takes as input model and text sample, prints text completions
    """
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    out = model.generate(
        **input_ids, max_length=200, eos_token_id=tokenizer.eos_token_id
    )

    print(tokenizer.decode(out[0], skip_special_tokens=True))

## Simple instruction prompt for text completion

Pass a start of text to the model and call ```model.generate``` method to complete the input text.


In [ ]:
prompt = "Describe London in 3 sentences"

generate_text(model, prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Describe London in 3 sentences.
London is a vibrant and diverse city that offers a wide range of cultural, historical, and entertainment attractions. From the iconic landmarks like Buckingham Palace and the Tower of London, to the bustling streets of Soho and Camden, London has something for everyone. Whether you're interested in exploring the city's rich history, enjoying its vibrant nightlife, or simply taking in the sights and sounds of this great metropolis, London is a must-visit destination.


Model ouputs some reasonable completion.

## Instruction prompt with mode detailed instruction

But what if we want model to generate some specific completion. E.g. what if we want model to list some of the best parks in London? Let's enrich our prompt with this instruction.

In [ ]:
instruction_prompt = (
    f"""Describe London in 3 sentences. Focus on the most famous parks in London."""
)

In [ ]:
print(instruction_prompt)

Describe London in 3 sentences. Focus on the most famous parks in London.


In [ ]:
generate_text(model, instruction_prompt)

Describe London in 3 sentences. Focus on the most famous parks in London.
London is a bustling metropolis with a rich history and culture, home to some of the most famous parks in the world. Hyde Park, one of the largest green spaces in the city, is a popular destination for picnics, boating, and people-watching. Regent's Park, known for its beautiful gardens and diverse wildlife, is another must-visit destination for nature lovers and fans of the famous Regent's Park Zoo.


Good. Now we see that model takes into accout our instuction and follows it outputting relevalant result.

## From Zero shot inference to one shot and few shot inference

Sometimes even with an instruction provided model fails to understand what is expected from it. In that case one can add to the prompt text example of desired input and output. Model can learn from this example and make more presice responses for new examples.

E.g. if we have a prompt `default_prompt = What is the sentiment of the following passage: "I liked this film"?` model may not understand what should be the desired output. We call such prompt a zero shot prompt. However we can enrich this prompt with one or more examples.

`One_shot_prompt = The sentiment for "I liked this film" is Positive.
Write a sentiment for the "I didn't like the film, it had bad acting"`

It is called one shot inference. Similarly, you can pass more than one examples and it will be few-shot.

Lets ask model to predict sentiment of the text. With zero-shot prompt we see that model returns correct sentiment but the output also contains options that model was considering that is useless for us. Let' use one or few shot inference to show the model some input/output examples so that model understood the context and expected output format better.

In [ ]:
default_prompt = """
What is the sentiment of the following passage: "I liked this film" ?
"""
generate_text(model, default_prompt)


What is the sentiment of the following passage: "I liked this film" ?

A) Neutral
B) Positive
C) Negative
D) Uncertain

Answer: B) Positive


In [ ]:
input_ids = tokenizer(default_prompt, return_tensors="pt").to("cuda")

out = model.generate(
    **input_ids,
    max_length=200,
    eos_token_id=tokenizer.eos_token_id,
    return_dict_in_generate=True,
    output_scores=True
)

We will create a dataframe with several labelled examples for sentiment classification problem.

In [ ]:
data = {
    "I liked this film": "Positive",
    "I didn't like the film, it had bad acting": "Negative",
    "I will never watch it again": "Negative",
    "My friend said it is 10/10 it and I think so.": "Positive",
}

In [ ]:
df = pd.DataFrame(data.items(), columns=["text", "label"])

In [ ]:
df

,text,label
0,I liked this film,Positive
1,"I didn't like the film, it had bad acting",Negative
2,I will never watch it again,Negative
3,My friend said it is 10/10 it and I think so.,Positive


Next let's create helper functions to create one shot and few shot prompts.

In [ ]:
def make_sentiment_instruction(sample_text: str, sample_label: str) -> str:
    """
    Wraps input text and its sentiment into instruction prompt
    """
    return f"The sentiment for '{sample_text}' is {sample_label}.\n"


def make_few_shot_prompt(
    examples_data: pd.DataFrame, example_to_predict: str, n_shots: int = 1
) -> str:
    """
    Takes as input data with sampled and a single sample to predict.
    Takes several samples from data and add them to a prompt as few shot examples for the model.
    Model uses these few shot examples to make a prediction.
    """
    few_shot_prompt = ""  # create empty string for our final prompt
    examples_data = examples_data.sample(
        n=n_shots, random_state=42
    )  # take random n_shots samples from our examples data
    for _, row in examples_data.iterrows():  # iterate over each example
        text, label = row["text"], row["label"]  # extract text and corresponding label
        few_shot_prompt += make_sentiment_instruction(
            text, label
        )  # make instruction and add it to our final prompt
    few_shot_prompt += f'Write a sentiment for the"{example_to_predict}"'  # add instruction to our example to predict in the end
    return few_shot_prompt

Example with one shot prompt with negative sentiment pediction

In [ ]:
neg_example_to_predict = "this was the worst experience ever."

one_shot_prompt = make_few_shot_prompt(df, neg_example_to_predict, 1)

generate_text(model, one_shot_prompt)

The sentiment for 'I didn't like the film, it had bad acting' is Negative.
Write a sentiment for the"this was the worst experience ever."
The sentiment for "this was the worst experience ever" is Negative.


Example with one shot prompt with positive sentiment pediction

In [ ]:
pos_example_to_predict = "I think it was really good."

one_shot_prompt = make_few_shot_prompt(df, pos_example_to_predict, 1)

generate_text(model, one_shot_prompt)

The sentiment for 'I didn't like the film, it had bad acting' is Negative.
Write a sentiment for the"I think it was really good."
The sentiment for 'I think it was really good' is Positive.


A few shot prompt with 3 input examples and a more complicated prediction sample.

In [ ]:
example_to_predict = "At first I was very skeptical but in the end i liked it very much"

few_shot_prompt = make_few_shot_prompt(df, example_to_predict, 3)

generate_text(model, few_shot_prompt)

The sentiment for 'I didn't like the film, it had bad acting' is Negative.
Write a sentiment for the"At first I was very skeptical but in the end i liked it very much" sentence.
The sentiment for 'At first I was very skeptical but in the end i liked it very much' is Positive.


## Text generation parameters exploration

you are given an input prompt, runa generative model to continue given text. In order to do so you would need to chose a model and call ```model.generate``` method. We offer you to play with text generation and understand how differen paratemers impact resulting generated text.

Some paramaters of ```model.generate``` method you may try to tune: top_k, top_p, num_beams, max_new_tokens, do_sample, tempreature

Read more about these params in this blogpost by HuggingFace
https://huggingface.co/blog/how-to-generate

One more source https://huggingface.co/docs/transformers/main_classes/text_generation


In [ ]:
prompt = "A step by step recipe to make bolognese pasta: "


def generate_with_params(model: PretrainedModel, tokenizer: tp.Union[PreTrainedTokenizer, PreTrainedTokenizerFast], prompt: str, gen_params: dict) -> None:
  """
  takes as input initialised model and tokenizer, a prompt and a custom congig with generation options.
  """
    model_inputs = tokenizer(prompt, return_tensors='pt').to('cuda')

    output = model.generate(
        **model_inputs,
        **gen_params
    )

    print("Output:\n" + 100 * '-')
    print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
generate_text(model, prompt)  # greedy decoding

A step by step recipe to make bolognese pasta: 

Bolognese pasta is a classic Italian dish that is made with ground beef, tomatoes, onions, carrots, celery, red wine, and beef broth. It is a hearty and flavorful sauce that is perfect for serving with pasta. Here is a step-by-step recipe for making bolognese pasta:

Ingredients:

* 1 pound ground beef
* 1 onion, finely chopped
* 2 cloves of garlic, minced
* 1 carrot, finely chopped
* 1 celery stalk, finely chopped
* 1 can of diced tomatoes
* 1 cup of red wine
* 4 cups of beef broth
* Salt and pepper,


In [ ]:
torch.cuda.empty_cache()

In [ ]:
prompt = "What can I expect from a workshop about large language models?"

gen_params = {
    "do_sample": True,
    "temperature": 0.9,
    "max_new_tokens": 200,
}
# example with high temp

generate_with_params(model, tokenizer, prompt, gen_params)

Output:
----------------------------------------------------------------------------------------------------
What can I expect from a workshop about large language models?

Large language models are a rapidly expanding area of study in natural language processing. These models have been trained on vast amounts of text data and are capable of generating text, answering questions, and even creating new text based on a given prompt. A workshop about large language models will likely cover a variety of topics, including:

1. Introduction to large language models: The workshop will start with an introduction to large language models, including their definition, types, and applications.
2. Architecture of large language models: The workshop will cover the different architectures used in large language models, such as transformer-based models and recurrent neural network-based models.
3. Training large language models: The workshop will discuss the different techniques used to train large lan

In [ ]:
gen_params = {
    "do_sample": True,
    "temperature": 0.1,
    "max_new_tokens": 200,
}
# example with low temp

generate_with_params(model, tokenizer, prompt, gen_params)

Output:
----------------------------------------------------------------------------------------------------
What can I expect from a workshop about large language models?

Large language models (LLMs) are a class of artificial intelligence models that are trained on vast amounts of text data to generate language outputs that are coherent and natural-sounding. These models have been increasingly used in a variety of applications, including language translation, text summarization, and language generation.

A workshop about LLMs could cover a range of topics, including:

1. Introduction to LLMs: This section of the workshop could provide an overview of what LLMs are, how they work, and their potential applications.
2. Types of LLMs: There are several types of LLMs, including generative models, discriminative models, and hybrid models. The workshop could discuss the strengths and weaknesses of each type of model.
3. Training LLMs: The workshop could cover the process of training LLMs, in

In [ ]:
gen_params = {"top_k": 50, "max_new_tokens": 200, "temperature": 0.3, "do_sample": True}

generate_with_params(prompt, gen_params)

passed params are: {'top_k': 10, 'max_new_tokens': 200, 'temperature': 0.5, 'do_sample': True}
A step by step recipe to make bolognese pasta: 

Bolognese pasta is a hearty, flavorful dish that originated in Bologna, Italy. The dish is made with ground beef, pork, or a combination of the two, simmered with tomatoes, onions, carrots, celery, red wine, and herbs. Here's a step-by-step recipe for making bolognese pasta at home:

Ingredients:

* 1 lb ground beef
* 1/2 lb ground pork
* 1 large onion, finely chopped
* 2 cloves of garlic, minced
* 2 carrots, finely chopped
* 2 stalks of celery, finely chopped
* 1 can of crushed tomatoes
* 1 cup of red wine
* 1 tbsp tomato paste
* 
